In [1]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn import preprocessing

In [7]:
df = pd.read_csv("data_details/trialPromoResults_onehot.csv")

In [8]:
df = df.drop(["index"], axis=1)
y = df["decision"]
print('Original dataset shape {}'.format(Counter(y)))

Original dataset shape Counter({'None': 828, 'A': 133, 'B': 39})


In [9]:
df.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans,decision
0,1,0,1,0,0,0,56.82,1,0,0,...,0,0,1,0,0,0,3105.39,33003.48,1776.81,None
1,0,1,0,1,0,0,87.35,3,0,0,...,0,1,0,1,0,0,4874.08,18941.99,863.56,None
2,0,1,0,0,1,0,28.75,0,0,0,...,0,0,0,0,0,1,14232.37,30013.32,3231.14,B
3,1,0,1,0,0,0,35.71,0,0,1,...,0,0,0,0,1,0,3214.93,15423.24,1996.09,None
4,0,1,0,0,1,0,20.53,0,1,0,...,0,0,0,1,0,0,3214.93,15423.24,1996.09,None


In [10]:
X = df.loc[:, df.columns != "decision"]
X.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,manuf,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans
0,1,0,1,0,0,0,56.82,1,0,0,...,0,0,0,1,0,0,0,3105.39,33003.48,1776.81
1,0,1,0,1,0,0,87.35,3,0,0,...,0,0,1,0,1,0,0,4874.08,18941.99,863.56
2,0,1,0,0,1,0,28.75,0,0,0,...,1,0,0,0,0,0,1,14232.37,30013.32,3231.14
3,1,0,1,0,0,0,35.71,0,0,1,...,0,0,0,0,0,1,0,3214.93,15423.24,1996.09
4,0,1,0,0,1,0,20.53,0,1,0,...,0,0,0,0,1,0,0,3214.93,15423.24,1996.09


### Run the SMOTE algorithm to balance the output classes as mentioned [here](http://contrib.scikit-learn.org/imbalanced-learn/stable/generated/imblearn.over_sampling.SMOTE.html)

In [11]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X, y)

In [12]:
print('Resampled dataset shape {}'.format(Counter(y_res)))

Resampled dataset shape Counter({'None': 828, 'B': 828, 'A': 828})


### So SMOTE in Python cannot yet handle categorical data - refer [this](https://stackoverflow.com/questions/47655813/oversampling-smote-for-binary-and-categorical-data-in-python)